In [1]:
pip install pillow numpy


In [1]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

# Path to the Noto Sans Devanagari TTF file
font_path = "Akshar_Unicode.ttf"
output_dir = "hindi_characters"

# Unicode range for Devanagari characters
start_code = 0x0900
end_code = 0x097F

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the font
font = ImageFont.truetype(font_path, 48)

for codepoint in range(start_code, end_code + 1):
    char = chr(codepoint)
    image = Image.new("L", (64, 64), "white")
    draw = ImageDraw.Draw(image)
    width, height = draw.textsize(char, font=font)
    draw.text(((64 - width) / 2, (64 - height) / 2), char, font=font, fill="black")
    image_path = os.path.join(output_dir, f"{codepoint}.png")
    image.save(image_path)

print("Dataset created successfully.")


<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.

Dataset created successfully.


<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width, height = draw.textsize(char, font=font)
<ipython-input-1-914a7e8256b5>:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.

In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class HindiCharacterDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        return image

# Define transformations
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the dataset
dataset = HindiCharacterDataset(root_dir='hindi_characters', transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Example of iterating through the dataloader
for images in dataloader:
    print(images.shape)  # Should output [batch_size, 1, 28, 28]


torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Generator and Discriminator networks
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input.view(-1, 784))

# Initialize models, loss function and optimizers
netG = Generator()
netD = Discriminator()
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0002)
optimizerG = optim.Adam(netG.parameters(), lr=0.0002)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # Update Discriminator
        netD.zero_grad()
        real = data
        batch_size = real.size(0)
        label = torch.full((batch_size,), 1.0)  # Convert to float
        output = netD(real).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()

        noise = torch.randn(batch_size, 100)
        fake = netG(noise)
        label.fill_(0.0)  # Convert to float
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        optimizerD.step()

        # Update Generator
        netG.zero_grad()
        label.fill_(1.0)  # Convert to float
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()

    print(f'Epoch [{epoch+1}/{num_epochs}] Loss D: {errD_real+errD_fake}, loss G: {errG}')

print("Training finished.")


Epoch [1/50] Loss D: 1.2151012420654297, loss G: 0.7070521116256714
Epoch [2/50] Loss D: 0.9860949516296387, loss G: 0.680450975894928
Epoch [3/50] Loss D: 0.8663981556892395, loss G: 0.6403652429580688
Epoch [4/50] Loss D: 0.869218111038208, loss G: 0.5765827894210815
Epoch [5/50] Loss D: 0.972034752368927, loss G: 0.4928649961948395
Epoch [6/50] Loss D: 1.1414263248443604, loss G: 0.4105813801288605
Epoch [7/50] Loss D: 1.3356289863586426, loss G: 0.3466895818710327
Epoch [8/50] Loss D: 1.4798024892807007, loss G: 0.3185352683067322
Epoch [9/50] Loss D: 1.470077633857727, loss G: 0.3495830297470093
Epoch [10/50] Loss D: 1.3425517082214355, loss G: 0.44460269808769226
Epoch [11/50] Loss D: 1.1173889636993408, loss G: 0.6470021605491638
Epoch [12/50] Loss D: 0.9380836486816406, loss G: 0.9574638605117798
Epoch [13/50] Loss D: 0.9093284606933594, loss G: 1.2870150804519653
Epoch [14/50] Loss D: 0.952337384223938, loss G: 1.3896853923797607
Epoch [15/50] Loss D: 0.9693425297737122, loss 

In [4]:
import torch
from PIL import Image
import numpy as np

# Define function to generate and save images using the Generator
def generate_and_save_images(generator, num_images, output_dir):
    noise = torch.randn(num_images, 100)
    fake_images = generator(noise).detach().cpu()

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in range(num_images):
        img = fake_images[i].squeeze().numpy()
        img = ((img + 1) * 127.5).astype(np.uint8)  # Convert to uint8
        img = Image.fromarray(img, 'L')
        img.save(os.path.join(output_dir, f"char_{i}.png"))

# Generate and save images
generate_and_save_images(netG, 80, 'generated_hindi_characters_2')


In [5]:
!apt-get install potrace -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpotrace0
Suggested packages:
  netpbm
The following NEW packages will be installed:
  libpotrace0 potrace
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 89.0 kB of archives.
After this operation, 246 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libpotrace0 amd64 1.16-2 [17.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 potrace amd64 1.16-2 [71.8 kB]
Fetched 89.0 kB in 1s (109 kB/s)
Selecting previously unselected package libpotrace0:amd64.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../libpotrace0_1.16-2_amd64.deb ...
Unpacking libpotrace0:amd64 (1.16-2) ...
Selecting previously unselected package potrace.
Preparing to unpack .../potrace_1.16-2_amd64.deb ...
Unpacking potrace (1.16

In [6]:
!pip install svgwrite fonttools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 881.5 kB/s eta 0:00:00


In [7]:
!pip install svgwrite fonttools


In [8]:
!pip install svgwrite fonttools scikit-image


In [23]:
import os
from PIL import Image
from fontTools.ttLib import TTFont, newTable
from fontTools.pens.ttGlyphPen import TTGlyphPen
import numpy as np
from skimage import measure

def image_to_glyph(image_path):
    image = Image.open(image_path).convert("L")
    image = image.resize((1000, 1000), Image.Resampling.LANCZOS)
    image = np.array(image)

    height, width = image.shape
    contours = measure.find_contours(image, 0.5)

    pen = TTGlyphPen(None)
    for contour in contours:
        pen.moveTo(tuple(contour[0]))
        for point in contour[1:]:
            pen.lineTo(tuple(point))
        pen.closePath()

    return pen.glyph()

def create_font_from_images(image_dir, output_font_path):
    font = TTFont()
    font['head'] = newTable('head')
    font['hhea'] = newTable('hhea')
    font['maxp'] = newTable('maxp')
    font['OS/2'] = newTable('OS/2')
    font['name'] = newTable('name')
    font['cmap'] = newTable('cmap')
    font['post'] = newTable('post')
    font['glyf'] = newTable('glyf')
    font['loca'] = newTable('loca')
    font['hmtx'] = newTable('hmtx')

    glyph_order = ['.notdef']
    cmap = {0: '.notdef'}
    font['glyf'].glyphs = {}
    font['hmtx'].metrics = {}

    for filename in os.listdir(image_dir):
        if filename.endswith('.png'):
            char_code = int(filename.split('_')[1].split('.')[0], 16)
            glyph_name = f'uni{char_code:04X}'
            image_path = os.path.join(image_dir, filename)

            glyph = image_to_glyph(image_path)
            font['glyf'].glyphs[glyph_name] = glyph
            font['hmtx'].metrics[glyph_name] = (1000, 0)  # Assuming fixed width and no LSB

            glyph_order.append(glyph_name)
            cmap[char_code] = glyph_name

    font.setGlyphOrder(glyph_order)
    font['cmap'].tables = [newTable('cmap_table')]
    font['cmap'].tables[0].format = 4
    font['cmap'].tables[0].platformID = 3
    font['cmap'].tables[0].platEncID = 1
    font['cmap'].tables[0].language = 0
    font['cmap'].tables[0].cmap = cmap

    font.save(output_font_path)
    print(f"Font generated at {output_font_path}")

# Create the font file from images
image_dir = "generated_hindi_characters_2"
output_font_path = "generated_font.ttf"
create_font_from_images(image_dir, output_font_path)


AssertionError: tag should be 4 characters long

In [39]:
import os
from PIL import Image
from fontTools.ttLib import TTFont, newTable, getTableModule
from fontTools.pens.ttGlyphPen import TTGlyphPen
import numpy as np
from skimage import measure

def image_to_glyph(image_path):
    image = Image.open(image_path).convert("L")
    image = image.resize((1000, 1000), Image.Resampling.LANCZOS)
    image = np.array(image)

    height, width = image.shape
    contours = measure.find_contours(image, 0.5)

    pen = TTGlyphPen(None)
    for contour in contours:
        pen.moveTo(tuple(contour[0]))
        for point in contour[1:]:
            pen.lineTo(tuple(point))
        pen.closePath()

    return pen.glyph()

def create_font_from_images(image_dir, output_font_path):
    font = TTFont()
    font['head'] = newTable('head')
    font['head'].flags = 0x0002  # Set the flags attribute directly
    font['hhea'] = newTable('hhea')
    font['maxp'] = newTable('maxp')
    font['OS/2'] = newTable('OS/2')
    font['name'] = newTable('name')
    font['cmap'] = newTable('cmap')
    font['post'] = newTable('post')
    font['glyf'] = newTable('glyf')
    font['loca'] = newTable('loca')
    font['hmtx'] = newTable('hmtx')

    glyph_order = ['.notdef']
    cmap = {0: '.notdef'}
    font['glyf'].glyphs = {}
    font['hmtx'].metrics = {}

    # Add .notdef glyph
    pen = TTGlyphPen(None)
    pen.moveTo((0, 0))
    pen.lineTo((1000, 0))
    pen.lineTo((1000, 1000))
    pen.lineTo((0, 1000))
    pen.closePath()
    notdef_glyph = pen.glyph()
    font['glyf'].glyphs['.notdef'] = notdef_glyph
    font['hmtx'].metrics['.notdef'] = (1000, 0)

    for filename in os.listdir(image_dir):
        if filename.endswith('.png'):
            char_code = int(filename.split('_')[1].split('.')[0], 16)
            glyph_name = f'uni{char_code:04X}'
            image_path = os.path.join(image_dir, filename)

            glyph = image_to_glyph(image_path)
            font['glyf'].glyphs[glyph_name] = glyph
            font['hmtx'].metrics[glyph_name] = (1000, 0)  # Assuming fixed width and no LSB

            glyph_order.append(glyph_name)
            cmap[char_code] = glyph_name

    font.setGlyphOrder(glyph_order)
    cmap_table = getTableModule('cmap').table__c_m_a_p()
    cmap_subtable = getTableModule('cmap').cmap_format_4(4)  # Specify the format argument
    cmap_subtable.platformID = 3
    cmap_subtable.platEncID = 1
    cmap_subtable.language = 0
    cmap_subtable.cmap = cmap

    cmap_table.tables = [cmap_subtable]
    font['cmap'] = cmap_table

    # Set the numGlyphs attribute for the maxp table
    font['maxp'].tableVersion = 0x00010000
    font['maxp'].maxZones = 1  # Set the maxZones attribute
    font['maxp'].numGlyphs = len(font.getGlyphOrder())

    # Save the font file
    font.save(output_font_path)
    print(f"Font generated at {output_font_path}")

# Create the font file from images
image_dir = "generated_hindi_characters_2"
output_font_path = "generated_font.ttf"
create_font_from_images(image_dir, output_font_path)


KeyError: 'maxTwilightPoints'